# Virtualized Patient Population

To-Do:
- re-import 'INSTANCE_OF' relationships between concepts, labs, and prescriptions
- Create OCCURS_WITH relationships between entities of interest that have z-scores or odds ratios
- Figure out how to add timedeltas between problems and the change in z-scores for labs or the occurrance of prescriptions

Entities of interest:
- Caregivers
- Patients
- Problem
- Prescriptions -> Concept
- Labevents -> D_Labitems -> Concept
- Diagnoses_Icd -> D_Icd_Diagnoses -> Concept (timedelta of limited utility, since ICD codes pertain to entire admission)
- Procedures_Icd -> D_Icd_Procedures -> Concept (timedelta of limited utility, since ICD codes pertain to entire admission)

Relationships to create between entities of interest:
![virtualized relationships](images/Virtual_relationship_schema.png)

We'll start with these relationships to improve performance on our current use cases:  
(:Problem) - [:INSTANCE_OF] -> (:Concept) - [:OCCURS_WITH {odds_ratio: __, source: 'MIMIC-III v1.4', updated: timestamp}] - (:Concept) <- [:INSTANCE_OF] - (:Prescriptions)  
(:Problem) - [:INSTANCE_OF] -> (:Concept) - [:OCCURS_WITH {odds_ratio: __, z_score: __, source: 'MIMIC-III v1.4', updated: timestamp}] - (:Concept) <- [:INSTANCE_OF] - (:Labevents)  
  
Looking ahead to how we'll use these algorithms, here are the queries I would like to write:


In [ ]:
def LikelyPrescriptions(cui_prob_list):
        
    query = '''
    MATCH (prob1:Problem)-[:INSTANCE_OF]->(:Concept)-[r:OCCURS_WITH]-(c:Concept)<-[:INSTANCE_OF]-(:Prescriptions)
    WHERE prob1.cui in {cui_prob_list} AND r.odds_ratio > 1.5
    RETURN c.term as Prescription, r.odds_ratio as `OddsRatio`
    ORDER BY OddsRatio DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    LikelyPrescriptions = session.run(query)
    LikelyPrescriptions = pd.DataFrame([dict(record) for record in LikelyPrescriptions])
    
    return LikelyPrescriptions

In [ ]:
def LikelyAbnormalLabs(cui_prob_list):
        
    query = '''
    MATCH (prob1:Problem)-[:INSTANCE_OF]->(:Concept)-[r:OCCURS_WITH]-(c:Concept)<-[:INSTANCE_OF]-(:Labevents)
    WHERE prob1.cui in {cui_prob_list} AND OddsRatio > 1.5
    RETURN c.term as Lab, r.odds_ratio as `OddsAbnormal`, r.z_score as `ZScore`
    ORDER BY OddsAbnormal DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    LikelyAbnormalLabs = session.run(query)
    LikelyAbnormalLabs = pd.DataFrame([dict(record) for record in LikelyAbnormalLabs])
    
    return LikelyAbnormalLabs